In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from main import TradingBot
import random
from IPython.display import clear_output

tb = TradingBot()

In [3]:
tb.loadData(timePeriod='2y', all=True, forceUpdate=True)
tb.computeSignal()

100%|██████████| 52/52 [00:49<00:00,  1.05it/s]


In [ ]:
tickerList = list(tb.data.keys())
keepgoing = True
maxProfit = 0
maxLoss = 0
totalRRR = 0
tradeCount = 0
balance = 0

while keepgoing:
    tradeCount += 1
    clear_output(wait=False)
    
    randomTicker = tickerList[random.randint(0, len(tickerList)-1)]
    indicator = tb.indicatorCollection[randomTicker]['sri']
    randomCandleIndex = random.randint(0+30, len(indicator.df.index)-31)
    indicator.showIndicator(randomCandleIndex)
    
    if not bool(float(input("trade?(1/0) : "))):
        tradeCount -= 1
        continue
    

    stoploss = float(input("stoploss : "))
    target = float(input("target : "))
    currentPrice = indicator.df.Close[randomCandleIndex]
    RRR = abs(target - currentPrice)/abs(stoploss - currentPrice)
    print(f'RRR = {RRR}')

    trade = 'long' if target > currentPrice else 'short'
    data = indicator.df

    result = 0
    start = randomCandleIndex+1
    end = randomCandleIndex + 30
    resultCandleIndex = start
    for i in range(start, end):
        if trade == 'long':
            if data.High[i] >= target:
                result = target - currentPrice
            elif data.Low[i] <= stoploss:
                result = stoploss - currentPrice
        else:
            if data.High[i] >= stoploss:
                result = currentPrice - stoploss
            elif data.Low[i] <= target:
                result = currentPrice - target
                
        if result > 0:
            result = result * (500/currentPrice)
            print(f"You've made a profit of Rs{result}")
            maxProfit = result if result > maxProfit else maxProfit
            totalRRR += RRR
            indicator.showIndicator(i)
            break
        elif result < 0:
            result = result * (500/currentPrice)
            print(f"You've booked a loss of Rs{abs(result)}")
            maxLoss = result if result < maxLoss else maxLoss
            totalRRR += RRR
            indicator.showIndicator(i)
            break
        else:
            resultCandleIndex = i
            continue

    if result == 0:
        print(f"No outcome after {end - start} iterations")
        tradeCount -= 1
        indicator.showIndicator(resultCandleIndex)
        
    balance += result
    
    keepgoing = bool(int(input("keep going?(1/0) : ")))
    
    
print(f"\nYour account balance after {tradeCount} trades is : {balance}")
print(f'\nBiggest Proft {maxProfit}')
print(f'\nBiggest Loss {maxLoss}')
print(f'\nAverage RRR {totalRRR/tradeCount}')